# Riskless lifetime spending

Here we figure out how to optimally spend a pot of money over a known time horizon, given access to a risk-free investment giving return $r_{ra}$ and a personal rate of time preference $r_{tp}$.

An optimal investment policy has these properties:

1. Spending more is better than spending less
1. There are decreasing marginal benefits to spending more
1. Spending should be as smooth as possible over time
1. Spending should react to changes in the value and quality of our investments, after tax and inflation-adjusted
1. Spending and investing should depend on our expected, but uncertain, personal longevity.

In [8]:
import numpy as np 
import polars as pl 
from polars import col 
from tqdm.autonotebook import tqdm
from findec import crra_utility

Imagine you are 65 years old, and you've just retired with $1M. Assume you know you're going to live for exactly 20 years. How should we spend our money such that we maximize our expected lifetime utility, over our remaining 20 years?

The solution is to spend a fraction $c_t$ of wealth $W_t$ which maximizes the sum of all future year's discounted utility.

I.e. choose a $c_t$ at every year $t$ such that we maximize

$$\sum_{t=1}^T \frac{U(c_t) W_t}{(1+r_{tp})^t}$$

where $W_t = W_{t-1}(1-c_{t-1})(1+r_{ra})$

In [2]:
years = np.linspace(1, 20, 20)
years

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20.])

In [3]:
initial_wealth = 1e6
GAMMA = 2.0
risk_adjusted_returns_per_year = 3e-2
future_utility_discount_rate_per_year = 1e-2
time_horizon = 20

In [12]:
def solve_consumption(
    *,
    W0: float,
    r_ra: float,
    r_tp: float,
    gamma: float,
    T: int,
    n_grid: int,
    c_grid_size: int,
):
    # 1. Build wealth grid
    W_max = W0 * (1 + r_ra) ** T * 2.0
    W_grid = np.linspace(1e-3, W_max, n_grid)

    # 2. Arrays for value and policy
    V = np.zeros((T + 2, n_grid))
    C = np.zeros((T + 2, n_grid))

    # 3. Terminal condition: V[T+1] = 0
    # Already zero by default. No bequest.

    # 4. Precompute discount factors
    discount_factors = [(1 + r_tp) ** t for t in range(T + 2)]

    # 5. Discretize c in [0, 1] but skip exact zero
    c_candidates = np.linspace(0, 1, c_grid_size)

    # 6. Backwards induction
    for t in tqdm(reversed(range(1, T + 1)), total=T):
        df = discount_factors[t]
        for i, W_now in enumerate(W_grid):
            best_value = -np.inf
            best_c = 0
            for c_frac in c_candidates:
                # consumption in dollars
                cons = c_frac * W_now
                # next period wealth
                W_next = (W_now - cons) * (1 + r_ra)

                # clamp W_next to grid range
                if W_next < W_grid[0]:
                    W_next_index = 0
                elif W_next > W_grid[-1]:
                    W_next_index = n_grid - 1
                else:
                    # approximate nearest index
                    W_next_index = np.searchsorted(W_grid, W_next)
                    if W_next_index >= n_grid:
                        W_next_index = n_grid - 1

                # immediate utility
                immediate_utility = crra_utility(cons, gamma=gamma)
                # discount that immediate utility
                discounted_utility = immediate_utility / df

                # future value
                future_value = V[t + 1, W_next_index]

                total_value = discounted_utility + future_value

                if total_value > best_value:
                    best_value = total_value
                    best_c = c_frac

                # store the best
                V[t, i] = best_value
                C[t, i] = best_c
    # 7. Grab the optimal consumption fraction at t=1 for W0
    i_closest = np.argmin(np.abs(W_grid - W0))
    optimal_c_init = C[1, i_closest]

    return C, V, W_grid, optimal_c_init

In [13]:
C, V, W_grid, optimal_c_init = solve_consumption(
    W0=initial_wealth,
    r_ra=risk_adjusted_returns_per_year,
    r_tp=future_utility_discount_rate_per_year,
    gamma=GAMMA,
    T=time_horizon,
    n_grid=2000,
    c_grid_size=1001,
)
print(f"Optimal c at t=1 for W0=${initial_wealth:,.2f}: {optimal_c_init:.4f}")


# Simulate forward
W_sim = [initial_wealth]
C_sim = []
for t in range(1, time_horizon + 1):
    i_closest = np.argmin(np.abs(W_grid - W_sim[-1]))
    c_star = C[t, i_closest]
    C_sim.append(c_star)
    cons = c_star * W_sim[-1]
    W_next = (W_sim[-1] - cons) * (1 + risk_adjusted_returns_per_year)
    W_sim.append(W_next)

print("\nYear | c*    | Wealth")
for year in range(time_horizon):
    print(f"{year+1:4d} | {C_sim[year]:.4f} | {W_sim[year]:.2f}")
print(f"Final wealth at year {time_horizon+1}: {W_sim[-1]:.2f}")

100%|██████████| 20/20 [02:06<00:00,  6.35s/it]

Optimal c at t=1 for W0=$1,000,000.00: 0.0640

Year | c*    | Wealth
   1 | 0.0640 | 1000000.00
   2 | 0.0670 | 964080.00
   3 | 0.0650 | 926471.24
   4 | 0.0700 | 892238.13
   5 | 0.0720 | 854674.90
   6 | 0.0740 | 816932.46
   7 | 0.0830 | 779173.84
   8 | 0.0910 | 735937.48
   9 | 0.0950 | 689036.19
  10 | 0.1000 | 642285.08
  11 | 0.1140 | 595398.27
  12 | 0.1290 | 543348.55
  13 | 0.1370 | 487454.29
  14 | 0.1540 | 433293.24
  15 | 0.1730 | 377563.07
  16 | 0.2090 | 321612.00
  17 | 0.2630 | 262026.94
  18 | 0.3380 | 198907.27
  19 | 0.5080 | 135626.91
  20 | 1.0000 | 68730.29
Final wealth at year 21: 0.00


Use dynamic programming to solve the following problem, with python.

I am 65 years old, and I have \$1M dollars. I can invest in a risk-free asset with returns of 3% after taz and above inflation. I discount my future utility of consumption by 2% per year. I have a constant relative risk aversion utility, with risk aversion parameter $\gamma = 2$. 

I spend $c_t$ fraction of my wealth every year. Choose a $c_t$ at every year $t$ such that we maximize

$$\sum_{t=1}^T \frac{U(c_t W_t)}{(1+r_{tp})^t}$$

where $W_t = W_{t-1}(1-c_{t-1})(1+r_{ra})$.